In [108]:
import os
import re
import csv
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin, urlparse
import requests

In [109]:
# Cấu hình đường dẫn
INPUT_FOLDER = "data"  
OUTPUT_FOLDER = "data_clean"  
URLS_CSV = "extracted_urls.csv"  

# Tạo thư mục output nếu chưa tồn tại
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

In [110]:
# HÀM LOẠI BỎ CÁC THÀNH PHẦN KHÔNG CẦN THIẾT
def remove_unwanted_elements(soup):

    # Tạo bản sao để không làm hỏng soup gốc
    clean_soup = BeautifulSoup(str(soup), 'html.parser')
    
    elements_to_remove = [
        'script', 'style', 'link', 'meta', 'noscript', 'video', 'audio'
        'footer', 'header', 'nav', 'aside', 'menu', 'img', 'button'
        'iframe', 'embed', 'object', 'ins', 'path', 'svg', 'symbol'
    ]
    
    for tag in elements_to_remove:
        for element in clean_soup.find_all(tag):
            element.decompose()

    return clean_soup

In [111]:
# HÀM TRÍCH XUẤT NỘI DUNG CHÍNH
def extract_main_content(soup):
    all_content = []

    text_elements = soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'li'])
    for elem in text_elements:
        text = elem.get_text(separator=' ', strip=True)
        all_content.append(text)
    
    return '\n'.join(all_content)

In [112]:
# HÀM CHỈNH SỬA TEXT
def normalize_text(text):
    if not text:
        return ""
    
    # Chuẩn hóa khoảng trắng
    text = re.sub(r'\s+', ' ', text)
    
    # Sửa lỗi dấu câu cơ bản
    text = re.sub(r'\s+([.,!?;:])', r'\1', text)  
    text = re.sub(r'([.,!?;:])(\w)', r'\1 \2', text)  
    
    # Loại bỏ ký tự đặc biệt, giữ lại dấu câu tiếng Việt
    text = re.sub(r'[^\w\s.,!?;:()\-ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠ-ỹ]', ' ', text)
    
    # chuyển về chữ thường
    text = text.lower()

    # Chuẩn hóa khoảng trắng lần cuối
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [113]:
# HÀM TRÍCH XUẤT URL LIÊN QUAN
def extract_related_urls(soup, base_domain="https://vnexpress.net"):
    urls = []
    
    for link in soup.find_all('a', href=True):
        url = link['href'].strip()
        
        # Lọc URL không mong muốn
        if (url and 
            not url.startswith(('javascript:', 'mailto:', 'tel:', '#', 'void(0)')) and
            len(url) > 5):
            
            # Chuẩn hóa URL
            if url.startswith('/'):
                url = urljoin(base_domain, url)
            elif not url.startswith(('http://', 'https://')):
                url = urljoin(base_domain + '/', url)
            
            # Kiểm tra URL hợp lệ
            if is_valid_url(url):
                urls.append(url)
    
    return list(set(urls))  # Loại bỏ trùng lặp

def is_valid_url(url):
    try:
        parsed = urlparse(url)
        return bool(parsed.netloc) and bool(parsed.scheme)
    except:
        return False

In [114]:
# HÀM XỬ LÝ FILE HTML CHÍNH
def process_html_file(html_file_path):

    try:
        # Đọc file HTML
        with open(html_file_path, 'r', encoding='utf-8', errors='ignore') as file:
            html_content = file.read()
        
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Lấy tiêu đề trang
        title = soup.title.string if soup.title else os.path.basename(html_file_path)
        
        clean_soup = remove_unwanted_elements(soup)
        
        # LƯU HTML ĐÃ LỌC VÀO THƯ MỤC html_clean
        html_clean_folder = "html_clean"
        os.makedirs(html_clean_folder, exist_ok=True)
        clean_html_filename = os.path.basename(html_file_path)
        clean_html_path = os.path.join(html_clean_folder, clean_html_filename)
        
        with open(clean_html_path, 'w', encoding='utf-8') as file:
            file.write(str(clean_soup))
        
        clean_text = extract_main_content(clean_soup)
        
        if clean_text and len(clean_text) > 0:
            
            clean_text = normalize_text(clean_text)
            
            urls = extract_related_urls(soup)
            
            return {
                'title': title,
                'clean_text': clean_text,
                'urls': urls,
                'clean_length': len(clean_text),
                'success': True,
            }
        else:
            return {
                'title': title,
                'clean_text': "",
                'urls': [],
                'clean_length': 0,
                'success': False,
            }
            
    except Exception as e:
        print(f"Lỗi xử lý {html_file_path}: {str(e)}")
        return {
            'title': os.path.basename(html_file_path),
            'clean_text': "",
            'urls': [],
            'clean_length': 0,
            'success': False,
        } 

In [115]:
# HÀM XỬ LÝ TẤT CẢ FILE VÀ LƯU KẾT QUẢ
def process_all_html_files():
    """
    Xử lý tất cả file HTML trong thư mục data
    """
    html_files = [f for f in os.listdir(INPUT_FOLDER) if f.endswith(('.html', '.htm'))]
    
    all_results = []
    url_data = []
    
    print(f"Bắt đầu xử lý {len(html_files)} file HTML...")
    print("=" * 60)
    
    for i, html_file in enumerate(html_files, 1):
        input_path = os.path.join(INPUT_FOLDER, html_file)
        output_filename = html_file.rsplit('.', 1)[0] + '.txt'
        output_path = os.path.join(OUTPUT_FOLDER, output_filename)
        
        print(f"[{i}/{len(html_files)}] Đang xử lý: {html_file}")
        
        result = process_html_file(input_path)
        
        # Lưu file TXT
        if result['success'] and result['clean_text']:
            with open(output_path, 'w', encoding='utf-8') as file:
                file.write(result['clean_text'])
        
        for url in result['urls']:
            url_data.append({
                'title': result['title'],
                'url': url,
                'source_file': html_file
            })

        
        all_results.append(result)
        
        print(f"Nội dung: {result['clean_length']} ký tự")
        print(f"URL: {len(result['urls'])} liên kết")
    
    # LƯU FILE CSV CHỨA URL LIÊN QUAN
    if url_data:
        df_urls = pd.DataFrame(url_data)
        df_urls.to_csv(URLS_CSV, index=False, encoding='utf-8')
        print(f"\nĐã lưu {len(url_data)} URL vào: {URLS_CSV}")
    
    return all_results

results = process_all_html_files()

Bắt đầu xử lý 407 file HTML...
[1/407] Đang xử lý: ao_Hon_Khoai_o_Ca_Mau_-_vung_bien_va_dai_at_phia_Tay_Nam_cua_To_quoc.html
Nội dung: 14457 ký tự
URL: 140 liên kết
[2/407] Đang xử lý: Kinh_nghiem_kham_pha_ho_a_Xanh_Vung_Tau_chi_tiet_nhat_2025.html
Nội dung: 8587 ký tự
URL: 104 liên kết
[3/407] Đang xử lý: Cam_nang_du_lich_Sam_Son.html
Nội dung: 11554 ký tự
URL: 68 liên kết
[4/407] Đang xử lý: Cam_nang_du_lich_Bao_Loc.html
Nội dung: 11357 ký tự
URL: 69 liên kết
[5/407] Đang xử lý: Bien_Ca_Na_Bai_bien_ep_nen_tho_cua_Phan_Rang.html
Nội dung: 11115 ký tự
URL: 60 liên kết
[6/407] Đang xử lý: Top_6_Bai_Bien_Quy_Nhon_ep_Ngat_Ngay_Khong_The_Bo_Qua.html
Nội dung: 10721 ký tự
URL: 68 liên kết
[7/407] Đang xử lý: Cam_nang_du_lich_Ly_Son.html
Nội dung: 11539 ký tự
URL: 75 liên kết
[8/407] Đang xử lý: Nui_Langbiang__Wikipedia_tieng_Viet.html
Nội dung: 9052 ký tự
URL: 278 liên kết
[9/407] Đang xử lý: Kham_pha_ve_ep_ao_xanh_Cu_Lao_Cham.html
Nội dung: 3745 ký tự
URL: 91 liên kết
[10/407] Đang xử lý: 

In [116]:
# BÁO CÁO KẾT QUẢ
def generate_report(results):
    success_count = sum(1 for r in results if r['success'])
    total_urls = sum(len(r['urls']) for r in results)
    total_clean_chars = sum(r['clean_length'] for r in results)
    
    print("\n" + "=" * 60)
    print("BÁO CÁO KẾT QUẢ XỬ LÝ")
    print("=" * 60)
    print(f"Tổng số file HTML: {len(results)}")
    print(f"File xử lý thành công: {success_count}")
    print(f"File thất bại: {len(results) - success_count}")
    print(f"Tổng số URL trích xuất: {total_urls}")
    print(f"Tổng ký tự văn bản sạch: {total_clean_chars:,}")
    
    # Hiển thị chi tiết 5 file đầu
    print(f"\nCHI TIẾT 5 FILE ĐẦU:")
    print("-" * 50)
    for i, result in enumerate(results[:5]):
        status = "success" if result['success'] else "fail"
        print(f"{status} {result['title'][:50]}...")
        print(f"Ký tự: {result['clean_length']} | URL: {len(result['urls'])}")

generate_report(results)


BÁO CÁO KẾT QUẢ XỬ LÝ
Tổng số file HTML: 407
File xử lý thành công: 407
File thất bại: 0
Tổng số URL trích xuất: 63089
Tổng ký tự văn bản sạch: 4,954,701

CHI TIẾT 5 FILE ĐẦU:
--------------------------------------------------
success Đảo Hòn Khoai ở Cà Mau - vùng biển và dải đất phía...
Ký tự: 14457 | URL: 140
success Kinh nghiệm khám phá hồ Đá Xanh Vũng Tàu chi tiết ...
Ký tự: 8587 | URL: 104
success Cẩm nang Du lịch SẦM SƠN 2025 từ A-Z: lưu trú, ăn ...
Ký tự: 11554 | URL: 68
success Cẩm nang Du lịch Bảo Lộc 2024 từ A-Z: Di chuyển, ă...
Ký tự: 11357 | URL: 69
success Biển Cà Ná: Bãi biển đẹp nên thơ của Phan Rang...
Ký tự: 11115 | URL: 60


In [117]:
# KIỂM TRA KẾT QUẢ
def verify_results():
    
    txt_files = [f for f in os.listdir(OUTPUT_FOLDER) if f.endswith('.txt')]
    
    print(f"\nKIỂM TRA KẾT QUẢ:")
    print(f"Số file TXT đã tạo: {len(txt_files)}")
    
    # Kiểm tra 3 file đầu
    for txt_file in txt_files[:3]:
        file_path = os.path.join(OUTPUT_FOLDER, txt_file)
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        print(f"\n{txt_file}:")
        print(f"Độ dài: {len(content)} ký tự")
        print(f"Nội dung: {content[:100]}...")

verify_results()


KIỂM TRA KẾT QUẢ:
Số file TXT đã tạo: 407

Cam_nang_du_lich_Tam_Coc_-_Bich_ong.txt:
Độ dài: 10402 ký tự
Nội dung: vne-go discover shorts podcasts thời sự chính trị kỷ nguyên mới 80 năm quốc khánh dân sinh việc làm ...

Cam_nang_du_lich_Chua_Tam_Chuc.txt:
Độ dài: 10983 ký tự
Nội dung: du lịch cẩm nang tam chúc 2 trở lại du lịch điều hướng nhanh đến tam chúc khi nào? di chuyển tham qu...

Cam_nang_du_lich_Ky_Co.txt:
Độ dài: 7492 ký tự
Nội dung: du lịch cẩm nang kỳ co 2 trở lại du lịch điều hướng nhanh di chuyển chơi gì ăn gì lưu ý bãi kỳ co th...
